In [1]:
import json

In [15]:
sample_candor_folder_path = '../data/candor/sample/0020a0c5-1658-4747-99c1-2839e736b481/'
sample_transcribe_output_path = '../data/candor/sample/0020a0c5-1658-4747-99c1-2839e736b481/transcription/transcribe_output.json'

In [16]:
with open(sample_transcribe_output_path) as f:
    out = json.load(f)

In [4]:
out['results'].keys()

dict_keys(['transcripts', 'channel_labels', 'items'])

In [5]:
# Without channel information
out['results']['items'][4]

{'start_time': '12.74',
 'end_time': '14.36',
 'alternatives': [{'confidence': '0.995', 'content': 'Just'}],
 'type': 'pronunciation'}

In [6]:
# With channel information
print(out['results']['channel_labels'].keys())
out['results']['channel_labels']['channels'][1]

dict_keys(['channels', 'number_of_channels'])


{'channel_label': 'ch_1',
 'items': [{'start_time': '198.34',
   'end_time': '198.66',
   'alternatives': [{'confidence': '0.911', 'content': 'Yeah'}],
   'type': 'pronunciation'},
  {'start_time': '200.74',
   'end_time': '201.01',
   'alternatives': [{'confidence': '0.974', 'content': 'hey'}],
   'type': 'pronunciation'},
  {'start_time': '201.01',
   'end_time': '201.16',
   'alternatives': [{'confidence': '0.976', 'content': "I'm"}],
   'type': 'pronunciation'},
  {'start_time': '201.16',
   'end_time': '201.76',
   'alternatives': [{'confidence': '0.65', 'content': 'gone'}],
   'type': 'pronunciation'},
  {'alternatives': [{'confidence': '0.0', 'content': '.'}],
   'type': 'punctuation'},
  {'start_time': '204.54',
   'end_time': '205.46',
   'alternatives': [{'confidence': '1.0', 'content': 'Yeah'}],
   'type': 'pronunciation'},
  {'start_time': '212.44',
   'end_time': '212.76',
   'alternatives': [{'confidence': '1.0', 'content': 'yeah'}],
   'type': 'pronunciation'},
  {'start

In [12]:
import sys
import os

scripts_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts'))

if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

from candor.create_raw_data import load_conversation_tokens

In [13]:
from pathlib import Path

In [19]:
# from ..scripts.candor.create_raw_data import load_conversation_tokens
# from pathlib import Path

In [17]:
convo_path = Path(sample_candor_folder_path)
convo_tokens = load_conversation_tokens(convo_path)

In [18]:
convo_tokens.loc[convo_tokens['type'] == 'pronunciation']

speaker  ...           type
conversation_id                      turn_id                            ...               
0020a0c5-1658-4747-99c1-2839e736b481 0        5fa072f4f4aa580b63834357  ...  pronunciation
                                     2        5fa072f4f4aa580b63834357  ...  pronunciation
                                     4        5fa072f4f4aa580b63834357  ...  pronunciation
                                     6        5fa072f4f4aa580b63834357  ...  pronunciation
                                     8        5fa072f4f4aa580b63834357  ...  pronunciation
...                                                                ...  ...            ...
                                     9025     5fa072f4f4aa580b63834357  ...  pronunciation
                                     9026     5fa072f4f4aa580b63834357  ...  pronunciation
                                     9028     5a73899f9cdd1800017786f0  ...  pronunciation
                                     9030     5a73899f9cdd1800017786f0  ...  pronunciation
                                     9032     5fa072f4f4aa580b63834357  ...  pronunciation

[7721 rows x 6 columns]

For my usecase, I want the backbiter transcripts (which separates out backchanneling by the interlocutor)

In [20]:
import pandas as pd

In [22]:
backbiter = pd.read_csv(convo_path / 'transcription/transcript_backbiter.csv')
backbiter

,turn_id,speaker,start,stop,utterance,backchannel,backchannel_count,backchannel_speaker,backchannel_start,backchannel_stop,interval,delta,questions,end_question,overlap,n_words
0,0,5fa072f4f4aa580b63834357,4.34,200.96,"Mhm. Mhm. Just, mm. And Uh huh, mm. Mhm. Mhm. ...",Yeah,1,5a73899f9cdd1800017786f0,198.34,198.66,NaN,196.62,2,False,False,39
1,1,5a73899f9cdd1800017786f0,200.74,201.76,hey I'm gone.,NaN,0,NaN,NaN,NaN,-0.22,1.02,0,False,True,4
2,2,5fa072f4f4aa580b63834357,202.14,212.75,"good, how are you? Yeah. Yeah, so this will be...",Yeah,1,5a73899f9cdd1800017786f0,204.54,205.46,0.38,10.61,1,False,False,14
3,3,5a73899f9cdd1800017786f0,212.44,213.28,yeah I've done a few,NaN,0,NaN,NaN,NaN,-0.31,0.84,0,False,True,6
4,4,5fa072f4f4aa580b63834357,213.24,213.28,I,NaN,0,NaN,NaN,NaN,-0.04,0.04,0,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,340,5fa072f4f4aa580b63834357,2704.89,2716.81,Oh man. Shoot. And I know this is your seventh...,Mhm. Yeah.,2,5a73899f9cdd1800017786f0,2706.29,2714.50,-0.61,11.92,1,False,True,35
341,341,5a73899f9cdd1800017786f0,2716.19,2716.42,Yeah.,NaN,0,NaN,NaN,NaN,-0.62,0.23,0,False,True,1
342,342,5fa072f4f4aa580b63834357,2716.82,2728.01,"well this has been really, really fun. This ha...",Oh yeah. Yeah.,3,5a73899f9cdd1800017786f0,2718.99,2726.51,0.40,11.19,0,False,False,41
343,343,5a73899f9cdd1800017786f0,2728.29,2731.92,Yeah I get to enjoy your day evening.,NaN,0,NaN,NaN,NaN,0.28,3.63,0,False,False,8


In [26]:
for utterance in backbiter['utterance']:
    print(utterance)

Mhm. Mhm. Just, mm. And Uh huh, mm. Mhm. Mhm. What? Mhm. Uh huh. Mhm. Yeah. Mhm. Mhm. Yeah. Oh. Oh yeah. Mhm. Yeah. Yeah. Great. Really? Yeah. Yeah. Yeah. Yeah. Yeah. Mhm. Yeah. Mhm. Yeah. Mhm. Yeah. Oh
hey I'm gone.
good, how are you? Yeah. Yeah, so this will be an interesting study. Mhm.
yeah I've done a few
I
of these before they're
don't
not
know.
too bad. Is this your first time doing?
this is actually my first one so uh yeah it's my first time doing yeah chad study so I guess I don't even know when there are, so it's kind of cool. There we go. Oh excellent,
yep
so hard that we've been going,
not too bad. Uh Feel a little crappy today but Uh huh it's that time of year I suppose
I don't
so
know.
yeah the kid's been a
no, are you getting sick? Oh
the kid hasn't been feeling good and I think she shared it with me so but
thanks, Tiny firm sponge. I swear that's that's what kids do, they're just kind of like you know, meant to sort of spread germs and viruses everywhere, but they're so

## Solving token to word mapping problem

In [67]:
# from transformers import GPT2Tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# ^ Must use tokenizer derived from PreTrainedTokenizerFast in order
# to have return_offset_mapping option
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", add_prefix_space=True)

In [68]:
import random

In [81]:
text_id = random.randint(0, len(backbiter) - 1)
text = backbiter['utterance'][text_id]
print(text)
print(backbiter['n_words'][text_id])

That's awesome. When I, like the last time I was in Wisconsin was like several years ago and one of the best meals that I had, there was actually at a fish fry, like a friday night fish fry. This place called the serb hall in Milwaukee and it was like the strangest environment to eaten ever. It was like very large event hall and all of the wait staff were like wearing very formal suits and tuxedos, But like the dress code of the actual people eating, there was very relaxed and when I say 40% of the people were wearing like exactly what you're wearing. I am not exaggerating. And so you were surrounded by like wait staff wearing like either tuxedos are very formal suits and then like green bay packers gear as far as the eye can see and the Munich selection ranged from like Sarah McLachlan to traditional Serbian folk music and like it was, it was very strange but the food was incredible and they also had like spotted cow as well too. So I had as much spotted cow because we don't have that

In [87]:
pretokenized_text = text.split()

In [88]:
inputs = tokenizer(
    pretokenized_text, 
    return_offsets_mapping=True, 
    add_special_tokens=False,
    is_split_into_words=True, 
)

In [91]:
tokens = inputs.tokens()
offset_mapping = inputs['offset_mapping']
print(offset_mapping)

for k, v in inputs.items():
    print(k, v)

# for token_id in range(len(inputs.tokens())):
#     print(token_id)

print(inputs.words())

for i, word in enumerate(pretokenized_text):
    print(f'{word=}, {inputs.word_to_tokens(i)}') 



[(0, 4), (4, 6), (0, 7), (7, 8), (0, 4), (0, 1), (1, 2), (0, 4), (0, 3), (0, 4), (0, 4), (0, 1), (0, 3), (0, 2), (0, 9), (0, 3), (0, 4), (0, 7), (0, 5), (0, 3), (0, 3), (0, 3), (0, 2), (0, 3), (0, 4), (0, 5), (0, 4), (0, 1), (0, 3), (3, 4), (0, 5), (0, 3), (0, 8), (0, 2), (0, 1), (0, 4), (0, 3), (3, 4), (0, 4), (0, 1), (0, 2), (2, 6), (0, 5), (0, 4), (0, 3), (3, 4), (0, 4), (0, 5), (0, 6), (0, 3), (0, 3), (3, 4), (0, 4), (0, 2), (0, 9), (0, 3), (0, 2), (0, 3), (0, 4), (0, 3), (0, 6), (6, 9), (0, 11), (0, 2), (0, 5), (0, 4), (4, 5), (0, 2), (0, 3), (0, 4), (0, 4), (0, 5), (0, 5), (0, 4), (0, 3), (0, 3), (0, 2), (0, 3), (0, 4), (0, 5), (0, 4), (0, 4), (0, 7), (0, 4), (0, 6), (0, 5), (0, 3), (0, 1), (1, 3), (3, 5), (5, 7), (7, 8), (0, 3), (0, 4), (0, 3), (0, 5), (0, 4), (0, 2), (0, 3), (0, 6), (0, 6), (0, 6), (6, 7), (0, 5), (0, 3), (0, 4), (0, 7), (0, 3), (0, 4), (0, 1), (0, 3), (0, 2), (2, 3), (0, 2), (0, 3), (0, 6), (0, 4), (0, 7), (0, 4), (0, 7), (0, 4), (0, 3), (3, 6), (0, 7), (7, 8)

In [92]:
for token in inputs.tokens():
    print(token)

ĠThat
's
Ġawesome
.
ĠWhen
ĠI
,
Ġlike
Ġthe
Ġlast
Ġtime
ĠI
Ġwas
Ġin
ĠWisconsin
Ġwas
Ġlike
Ġseveral
Ġyears
Ġago
Ġand
Ġone
Ġof
Ġthe
Ġbest
Ġmeals
Ġthat
ĠI
Ġhad
,
Ġthere
Ġwas
Ġactually
Ġat
Ġa
Ġfish
Ġfry
,
Ġlike
Ġa
Ġfr
iday
Ġnight
Ġfish
Ġfry
.
ĠThis
Ġplace
Ġcalled
Ġthe
Ġser
b
Ġhall
Ġin
ĠMilwaukee
Ġand
Ġit
Ġwas
Ġlike
Ġthe
Ġstrang
est
Ġenvironment
Ġto
Ġeaten
Ġever
.
ĠIt
Ġwas
Ġlike
Ġvery
Ġlarge
Ġevent
Ġhall
Ġand
Ġall
Ġof
Ġthe
Ġwait
Ġstaff
Ġwere
Ġlike
Ġwearing
Ġvery
Ġformal
Ġsuits
Ġand
Ġt
ux
ed
os
,
ĠBut
Ġlike
Ġthe
Ġdress
Ġcode
Ġof
Ġthe
Ġactual
Ġpeople
Ġeating
,
Ġthere
Ġwas
Ġvery
Ġrelaxed
Ġand
Ġwhen
ĠI
Ġsay
Ġ40
%
Ġof
Ġthe
Ġpeople
Ġwere
Ġwearing
Ġlike
Ġexactly
Ġwhat
Ġyou
're
Ġwearing
.
ĠI
Ġam
Ġnot
Ġexagger
ating
.
ĠAnd
Ġso
Ġyou
Ġwere
Ġsurrounded
Ġby
Ġlike
Ġwait
Ġstaff
Ġwearing
Ġlike
Ġeither
Ġt
ux
ed
os
Ġare
Ġvery
Ġformal
Ġsuits
Ġand
Ġthen
Ġlike
Ġgreen
Ġbay
Ġpack
ers
Ġgear
Ġas
Ġfar
Ġas
Ġthe
Ġeye
Ġcan
Ġsee
Ġand
Ġthe
ĠMunich
Ġselection
Ġranged
Ġfrom
Ġlike
ĠSarah
ĠMcL
ach
lan
Ġto
Ġtraditional
ĠSerbian